In [6]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from openpyxl.styles import PatternFill

train = pd.read_csv('train.csv')
train_pd = train.copy()
train_pd = train_pd.drop('id', axis=1)
enum_columns = [col for col in train_pd.columns if train_pd[col].dtype == 'float64' or train_pd[col].dtype == 'int64']
train_pd = train_pd[enum_columns]
print(train_pd.columns)

# Drop rows with NaN values in 'sii'
train_pd = train_pd.fillna(train_pd.mean())

mi_scores = {}


for col1 in train_pd.columns:
    for col2 in train_pd.columns:
        X = train_pd[[col1]].values
        y = train_pd[col2].values
        # Calculate mutual information
        # if train[col1].dtype == 'float64' or train[col1].dtype == 'float64':
        #     mi = mutual_info_regression(X, y, discrete_features=True)
        # elif train[col1].dtype == 'int64' or train[col1].dtype == 'int64':
        #     mi = mutual_info_classif(X, y, discrete_features=True)
        # else:
        #     mi = mutual_info_regression(X, y, discrete_features=True)
        mi = mutual_info_regression(X, y, discrete_features=True)
        mi_scores[(col1, col2)] = mi[0]

mi_scores = pd.Series(mi_scores)

mi_matrix = mi_scores.unstack().fillna(0)

# plt.figure(figsize=(20, 15))
# sns.heatmap(mi_matrix, annot=True, fmt=".2f", cmap='viridis', linewidths=0.5)
# plt.title('Mutual Information Heatmap', fontsize=20)
# plt.show()
wb = openpyxl.Workbook()
ws = wb.active

def get_color(value):
    red = 120 + int(20 * value)
    green = 220 - int(20 * value)
    blue = 220 - int(20 * value)
    return f"{red:02x}{green:02x}{blue:02x}"

# Apply the color to each cell based on its value
for r_idx, row in enumerate(mi_matrix.itertuples(), 1):
    for c_idx, value in enumerate(row[1:], 1):
        cell = ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)
        try:
            color = get_color(value)
        except:
            color = "FFFFFF"
        cell.fill = PatternFill(start_color=color, end_color=color, fill_type="solid")
# Write the column and index names
for c_idx, col_name in enumerate(mi_matrix.columns, 1):
    ws.cell(row=1, column=c_idx + 1, value=col_name)
for r_idx, idx_name in enumerate(mi_matrix.index, 1):
    ws.cell(row=r_idx + 1, column=1, value=idx_name)

# Round the values to 2 decimal places
for row in ws.iter_rows(min_row=2, min_col=2, max_row=ws.max_row, max_col=ws.max_column):
    for cell in row:
        cell.value = round(cell.value, 2)
# Save the workbook
wb.save('mi_matrix_colored.xlsx')

Index(['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
       'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
       'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone',
       'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
       'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03